In [ ]:
# journalist
# sociologist
# politicol consultants


#deliverables
# presenttion for a non technical audience
# readme markdown file telling your story OR jupyter notebook with markdown for a nontech audience
# script with helper functions
# a graph or two



In [1]:
import pyspark as ps    # for the pyspark suite

spark = ps.sql.SparkSession.builder \
            .master("local[2]") \
            .appName("Spark Session Intro") \
            .getOrCreate()

sc = spark.sparkContext 

In [2]:
sc.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.driver.port', '38641'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '1d74aa07c89c'),
 ('spark.app.id', 'local-1616722037801'),
 ('spark.sql.warehouse.dir',
  'file:/home/jovyan/work/RPP/CaseStudies/FrenchElectionTweets/spark-warehouse'),
 ('spark.executor.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.app.name', 'Spark Session Intro'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.startTime', '1616722036970'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'local[2]')]

In [4]:

frenchtweets_df = spark.read.json('data/french_tweets.json')

In [ ]:
yelp_business_df = yelp_df.createOrReplaceTempView('yelp_business')
out = yelp_df.filter((col('stars') == 5) &
                        (col('city') == 'Phoenix') &
                        (col('attributes.`Accepts Credit Cards`') == 'true') &
                        (array_contains(col('categories'),'Restaurants'))).select('name')
print(len([print(row) for row in out.collect()]))
print(out.collect().count())
print('___'*10)
print(out.show())